# AoC 2025 Day 5


In [ ]:

from aocd.models import Puzzle
from fasthtml.common import show

In [ ]:
puzzle = Puzzle(day=5, year=2025)
puzzle

<Puzzle(2025, 5) at 0x72ff5b2e7260 - Cafeteria>

In [ ]:
def prose(puzzle): return show(puzzle._get_prose(force_precheck=True))

In [ ]:
prose(puzzle)

## Part 1

- parse data done
- loop through ingreditents
- function is_fresh for each ingreditents make use make sure is at least contained in any of the ranges, if so is_fresh returns true
- sum all true values for all ingreditents


In [ ]:
exs=puzzle.examples[0].input_data
rngs, ings  = exs.split("\n\n")
rngs, ings



('3-5\n10-14\n16-20\n12-18', '1\n5\n8\n11\n17\n32')

In [ ]:
[tuple(r.split("-")) for r in rngs.splitlines()]
    

[('3', '5'), ('10', '14'), ('16', '20'), ('12', '18')]

In [ ]:
[tuple(map(int, r.split("-"))) for r in rngs.splitlines()]
    

[(3, 5), (10, 14), (16, 20), (12, 18)]

In [ ]:
def _parse_ranges(rngs): return [tuple(map(int, r.split("-"))) for r in rngs.splitlines()]


In [ ]:
list(map(int, ings.splitlines()))

[1, 5, 8, 11, 17, 32]

In [ ]:
def _parse_ingredients(ings): return list(map(int, ings.splitlines()))


In [ ]:
def parse_data(s):
    rngs,ings = s.strip().split('\n\n')
    return _parse_ranges(rngs), _parse_ingredients(ings)

parse_data(exs)

([(3, 5), (10, 14), (16, 20), (12, 18)], [1, 5, 8, 11, 17, 32])

In [ ]:
rngs, ings = parse_data(exs)
for i in ings: print(i, any(lo <= i <= hi for lo, hi in rngs))
        
            

1 False
5 True
8 False
11 True
17 True
32 False


In [ ]:
def is_fresh(x, rngs):
    "Check if an ingredient x is within any of the ranges in rngs"""
    return any(lo <= x <= hi for lo, hi in rngs)


In [ ]:
def solve(data):
    rngs, ings = parse_data(data)
    return sum(1 for x in ings if is_fresh(x, rngs))

solve(exs)

3

In [ ]:
solve(puzzle.input_data)

737

In [ ]:
puzzle.answer_a = 737

That's the right answer!  You are one gold star closer to decorating the North Pole. [Continue to Part Two]


## Part 2

Plan of attack

- for part 2 i think i just need to unravel the ranges into a set
- set will handle duplicates and the ranges will be unique


In [ ]:
rngs, _ = parse_data(puzzle.input_data)
max(hi-lo for lo, hi in rngs)


9141710840246

i think the issue with my approach will be the size of the set. This is not viable as i would run out of memory

In [ ]:
rngs, _ = parse_data(exs)
rngs.sort()
rngs

[(3, 5), (10, 14), (12, 18), (16, 20)]

In [ ]:
merged = []
for r1,r2 in zip(rngs, rngs[1:]):
    if r1[1] >= r2[0]- 1:  # -1 as (3,5) (6,8) is not a gap 
        merged.append((r1[0], r2[1]))
    else: merged.append(r1)

        

In [ ]:
merged

[(3, 5), (10, 18), (12, 20)]

Ok i think something is not correct here as im clearly missing some values in this approach, eg last value is not considered. lets discuss, dont reveal solution of methods just hint on what went wrong

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Good catch! You're noticing an important issue. Think about what happens with the *last* range in your original list when you use `zip(rngs, rngs[1:])`.

What does `zip` do when the two lists have different lengths? And which range from your original list never gets to be `r1` in your loop?

Last value will be missed, as zip used in this context is droping the last range. Also completely self contained ranges will not be removed with this method

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Exactly right on both points!

**First issue**: `zip(rngs, rngs[1:])` pairs up elements but stops when the shorter list ends, so the last range never gets processed.

**Second issue**: Your merging logic only looks at *consecutive pairs*. If you have a range that's completely contained within another (like `(12, 18)` inside `(10, 20)`), it might slip through depending on how the merging progresses.



In [ ]:
# need to define overlaps and non overlaps and buld the list
merged = []
for lo, hi in rngs:
    if not merged or lo > merged[-1][1] + 1:  # add if emptyor is the current range's start beyond the last merged range's end, no overlap, +1 for adjacent 
        merged.append((lo, hi)) 
    else: # Overlap! Extend the last range , keep current merged last val lo but recheck new hi
        merged[-1] = (merged[-1][0], max(hi, merged[-1][1]))

In [ ]:
merged


[(3, 5), (10, 20)]

In [ ]:
sum(hi-lo for lo, hi in merged)

12

In [ ]:
sum(hi-lo+1 for lo, hi in merged) # it's inclusive of top and bottom

14

In [ ]:
def solve_part2(data):
    rngs, _ = parse_data(data)
    rngs.sort() 
    merged = []
    for lo, hi in rngs:
        if not merged or lo > merged[-1][1] + 1: merged.append((lo, hi)) # empty or no overlap
        else: merged[-1] = (merged[-1][0], max(hi, merged[-1][1])) # Overlap or adjacent
    return sum(hi - lo + 1 for lo, hi in merged)

In [ ]:
solve_part2(exs)

14

In [ ]:
puzzle.answer_b = solve_part2(puzzle.input_data)

That's the right answer!  You are one gold star closer to decorating the North Pole.You have completed Day 5! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [ ]:
solve_part2(puzzle.input_data)

357485433193284

## Reflection

Part 1 was easy , Part 2 a bt more tricky i had to sit down and define with pen and paper what made an overlap and hat made a clean ranges to be added. Also i a few times was off by one. In the adjacent ranges and at the end when calculating the differences. a good brain teaser overall. 

I would investe on a better data structure for this problem. using indexes to represent high and lo points and ranges are not very readable 

Please comment on my reflection and also on the solutions used. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Great reflection! You've identified the key challenges well:

**Your insights:**
- Part 2's complexity jump was real—moving from "check membership" to "merge overlapping ranges" required careful thinking
- The off-by-one errors are classic! Inclusive ranges (`3-5` means 3, 4, *and* 5) are notorious for this
- Pen and paper for working out the overlap logic—that's exactly the right approach for interval problems

**On your solutions:**
- Your Part 1 is clean and readable. The `any()` with a generator is Pythonic and efficient
- Your Part 2 merge algorithm is solid—you correctly handle both overlaps and adjacent ranges with the `+1` check
- The sort-then-merge pattern you used is a classic algorithm for interval merging (O(n log n) time)

**On data structures:**
You mentioned wanting better readability than tuple indices. What would make it clearer for you? Named tuples? A simple `Range` class? Or something else?

Named tuples would be my choice. I like how these are instacniated as a class used type hints. Please provide a mock up of how part 2 would look with named tuples

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Here's how Part 2 would look with named tuples:

```python
from typing import NamedTuple

class Range(NamedTuple):
    start: int
    end: int

def solve_part2_named(data):
    rngs, _ = parse_data(data)
    rngs = [Range(lo, hi) for lo, hi in sorted(rngs)]
    
    merged = []
    for r in rngs:
        if not merged or r.start > merged[-1].end + 1:
            merged.append(r)
        else:
            merged[-1] = Range(merged[-1].start, max(r.end, merged[-1].end))
    
    return sum(r.end - r.start + 1 for r in merged)
```

Notice how `r.start` and `r.end` are much clearer than `lo` and `hi` or index access. The type hints also make it explicit what you're working with.

Would you like to try running this version to see if you prefer it?

yeah this was a problem i should've invested in a data struture that increases readability. i think your named tuple function is more readable. Please e critical

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

I'll be critical as requested!

**What's better:**
- `r.start` and `r.end` are self-documenting—no mental translation needed
- Type hints catch errors early if you accidentally pass wrong data
- The `Range` class makes the domain concept explicit

**What could be even better:**
- The `_parse_ranges` function should return `Range` objects directly, not tuples that get converted later
- You could add a method like `r.overlaps_or_adjacent(other)` to encapsulate that `r.start > merged[-1].end + 1` logic
- The `+1` for adjacency is still a bit magic—a comment or helper method would clarify intent

**A question for you:**
Do you think adding methods to `Range` (like `overlaps_or_adjacent` or `merge_with`) would help, or would that be over-engineering for a one-off puzzle?

maybe overengeneering as this is an aoc puzzle, but the overhead added just with a simple named tuple class makes the code readability increase a lot on my opinion. it would've been the best balance

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

I completely agree with you. The named tuple hits the sweet spot—minimal overhead, maximum readability gain. It's just a few lines but makes the intent crystal clear throughout.

For AoC, you want to move fast but also understand your code when debugging. The named tuple gives you both. Methods like `overlaps_or_adjacent` would probably slow you down more than help.

Good instinct on finding that balance. Would you like to refactor your actual solution with the `Range` named tuple, or are you ready to move on to the next puzzle?